In [1]:
!pip install git+https://github.com/cloudera/cmlextensions.git
!pip install ray[default]
!pip install logger

  Cloning https://github.com/cloudera/cmlextensions.git to /tmp/pip-req-build-u4_tonq6
  Running command git clone --filter=blob:none --quiet https://github.com/cloudera/cmlextensions.git /tmp/pip-req-build-u4_tonq6
  Resolved https://github.com/cloudera/cmlextensions.git to commit d30304e29ab636d238b612119c54648825fb149d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cmlextensions: filename=cmlextensions-0.1-py3-none-any.whl size=13977 sha256=dbb3ebc1997139e33b8bacda80443e79aed90ef79b12a291b519ff9052288c3a
  Stored in directory: /tmp/pip-ephem-wheel-cache-t009l9j8/wheels/b5/c5/e7/13fdd7b0727106324510c3eb8f64f2268a96e68f1f70079dd4
Successfully built cmlextensions
  Preparing metadata (setup.py) ... done
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1788 sha256=97728451dcc42bd2e5bfe7bf3461813204c16126264b1828f683e45c164379f1
  Stored in directory: /home/cds

In [2]:
!pip install -U ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/d4/17/8b2ce5765dd423433d2e0727712629c46152fb0bc706b0977f847480f262/ipywidgets-8.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.11 from https://files.pythonhosted.org/packages/93/c1/68423f43bc95d873d745bef8030ecf47cd67f932f20b3f7080a02cff43ca/widgetsnbextension-4.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.11 from https://files.pythonhosted.org/packages/65/f6/659ca44182c86f57977e946047c339c717745fda9f43b7ac47f274e86553/jupyterlab_widgets-3.0.11-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.4 MB/s eta 0:00:00:00:01


In [3]:
from cmlextensions.ray_cluster import RayCluster

In [4]:
cluster = RayCluster(num_workers=3, worker_cpu=2, worker_memory=4, worker_nvidia_gpu=1, head_cpu=2, head_memory=4, head_nvidia_gpu=0)
cluster.init()

Starting ray head...
Starting 3 ray workers...

--------------------
Ray cluster started
--------------------

The Ray dashboard is running at
https://5avval4ltqx7la5a.cml.apps.puncdsocpprd.os.net.ibm.com/

To connect to this Ray cluster from this CML Session,
use the following Python code:
  import ray
  ray.init(address='ray://10.129.8.142:10001')



In [5]:
import ray

if ray.is_initialized:
    ray.shutdown()

# GPUs and CPUs are auto-detected by ray when you call ray.init()
ray.init(address=cluster.get_client_url())
# -----> this doesn't work because get_gpu_ids cannot be used in a driver.
# print('ray.init()',ray.get_gpu_ids())

SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.9.16
Ray version:,2.9.2
Dashboard:,http://127.0.0.1:8090


In [6]:
import ray
ray.init()

2024-05-31 04:36:26,286	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=2.00gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-05-31 04:36:27,501	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8090 


Python version:,3.9.16
Ray version:,2.9.2
Dashboard:,http://127.0.0.1:8090


In [19]:
print('ray.init()',ray.get_gpu_ids())

ray.init() []


In [7]:
import time
from logger import logger
from multiprocessing import cpu_count
import os
import torch

@ray.remote(num_cpus = 1,num_gpus=1/100)
def f(i):
    print(ray.get_gpu_ids())
    # Here, CUDA_VISIBLE_DEVICES should be correctly set
    print(os.getenv("CUDA_VISIBLE_DEVICES"))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device,ray.get_gpu_ids())
    torch.rand(1, 1).to(device)
    time.sleep(1)
    print("@ray.remote","f",2,i)
    return i*i
bt = time.time()
futures = [f.remote(i) for i in range(20)]
logger.info(ray.get(futures))
et = time.time()
logger.info(f"time {et-bt}")

(f pid=518, ip=10.129.8.143) [0]
(f pid=518, ip=10.129.8.143) 0
(f pid=518, ip=10.129.8.143) cuda [0]
(f pid=488, ip=10.129.8.143) [0]
(f pid=488, ip=10.129.8.143) 0
(f pid=488, ip=10.129.8.143) cuda [0]
(f pid=486, ip=10.129.8.144) [0]
(f pid=486, ip=10.129.8.144) 0
(f pid=516, ip=10.129.8.144) [0]
(f pid=516, ip=10.129.8.144) 0
(f pid=486, ip=10.129.8.144) cuda [0]
(f pid=516, ip=10.129.8.144) cuda [0]
(f pid=518, ip=10.129.8.143) @ray.remote f 2 2
(f pid=488, ip=10.129.8.143) @ray.remote f 2 3
(f pid=486, ip=10.129.8.144) @ray.remote f 2 5
(f pid=516, ip=10.129.8.144) @ray.remote f 2 4
(f pid=515, ip=10.129.10.28) [0]
(f pid=515, ip=10.129.10.28) 0
(f pid=468, ip=10.129.10.28) [0]
(f pid=468, ip=10.129.10.28) 0
(f pid=515, ip=10.129.10.28) cuda [0]
(f pid=468, ip=10.129.10.28) cuda [0]
(f pid=468, ip=10.129.10.28) @ray.remote f 2 0
(f pid=515, ip=10.129.10.28) @ray.remote f 2 1
(f pid=654, ip=10.129.8.143) [0]
(f pid=654, ip=10.129.8.143) 0
(f pid=654, ip=10.129.8.143) cuda [0]
(f p

In [11]:
ray.shutdown()           # Stop Ray
cluster.terminate()      # Delete the cluster